In [ ]:
from google.colab import userdata
from openai import OpenAI

# Retrieve API key from Colab user data
api_key = userdata.get('Open_API_Key')
if not api_key:
    raise ValueError("Open_API_Key not found in user data. Please set it in Colab.")

# Initialize OpenAI client with API key
client = OpenAI(api_key=api_key)

In [5]:
import os
from IPython.display import display, Image  # display functions

# Use an existing assistant
assistant_id = "asst_f48G6l0owhKzgjGXV7O32EZ2"  # assistant's ID from playground

# Create a new thread for interaction
thread = client.beta.threads.create()

# Ensure a directory exists to save images
image_dir = "assistant_images"
os.makedirs(image_dir, exist_ok=True)

# Start the conversation loop
while True:
    # Step 1: Get user input
    user_input = input("User: ")  # Prompt for next input

    # Step 2: If the user types 'exit', break the loop and end the conversation
    if user_input.lower() == 'exit':
        print("Ending the session. Goodbye!")
        break

    # Step 3: Send the user input message to the assistant
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_input
    )

    # Step 4: Ask the assistant to process the task
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant_id,
        instructions="Analyze the data and respond based on user request."
    )

    # Step 5: Fetch only the first assistant response
    if run.status == 'completed':
        messages = client.beta.threads.messages.list(thread_id=thread.id, order="desc", limit=5)

        # Find and print the first assistant response only
        for msg in messages:
            if msg.role == 'assistant':
                first_content = msg.content[0]  # Get the first content block

                if hasattr(first_content, "text"):
                    # Handle text responses
                    print(f"Assistant: {first_content.text.value}")

                elif hasattr(first_content, "image_file"):
                    # Handle image responses
                    file_id = first_content.image_file.file_id
                    response = client.files.content(file_id)  # Get binary image content

                    # Save the image locally
                    image_path = os.path.join(image_dir, f"{file_id}.png")
                    with open(image_path, "wb") as f:
                        f.write(response.content)

                    print(f"Assistant sent an image:")
                    # Resize image by setting width (adjust value as needed)
                    display(Image(image_path, width=500))  # Width set to 500px for better readability

                else:
                    print("Assistant sent an unsupported response format.")

                break  # Stop after handling the first assistant response

    else:
        print("Assistant is processing your request. Please wait...")

    print("\n")  # Ensures a clean space before the next user prompt


User: how many accounts are targets?
Assistant: There are 311 accounts marked as targets in the dataset.


User: exit
Ending the session. Goodbye!


Updated Code:

In [ ]:
import os
import base64
import ipywidgets as widgets
import markdown
from IPython.display import display, HTML, Image, Javascript

# Use an existing assistant
assistant_id = "asst_f48G6l0owhKzgjGXV7O32EZ2"  # Assistant's ID from playground

# Create a new thread for interaction
thread = client.beta.threads.create()

def auto_focus_input():
    """Auto-focus input field after each response."""
    display(Javascript('document.getElementById("user_input").focus();'))

# Create input field and submit button
user_input_widget = widgets.Text(
    value='',
    placeholder='Type your message...',
    description='User:',
    layout=widgets.Layout(width='80%')
)

submit_button = widgets.Button(description="Submit")

# Output area for conversation
output_area = widgets.Output()

# Function to handle user submission
def handle_submit(_):
    user_input = user_input_widget.value.strip()  # Remove extra spaces

    if not user_input or len(user_input) < 2:  # Prevents accidental empty/single-letter inputs
        return

    # Clear input field
    user_input_widget.value = ""

    # Exit condition
    if user_input.lower() == 'exit':
        with output_area:
            print("\nEnding the session. Goodbye!")
        return

    # Display User's message (with bold "User:" text)
    with output_area:
        display(HTML(f"<b>User:</b> {markdown.markdown(user_input)}<br>"))

    # Send the user input to the assistant
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_input
    )

    # Process the assistant's response
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant_id,
        instructions="Analyze the data and respond based on user request."
    )

    # Fetch assistant's response
    if run.status == 'completed':
        messages = client.beta.threads.messages.list(thread_id=thread.id, order="desc", limit=5)

        with output_area:
            # Find the first assistant response
            for msg in messages:
                if msg.role == 'assistant':
                    text_responses = []
                    image_elements = []

                    for content in msg.content:
                        if hasattr(content, "text"):
                            # Process assistant's text response as Markdown
                            markdown_response = markdown.markdown(content.text.value.replace("\n", "<br>"))
                            text_responses.append(markdown_response)

                        elif hasattr(content, "image_file"):
                            # Handle image responses
                            file_id = content.image_file.file_id
                            response = client.files.content(file_id)  # Get binary image content
                            image_data = response.content

                            # Convert image to base64 for displaying & download
                            encoded_image = base64.b64encode(image_data).decode('utf-8')
                            image_html = f'<img src="data:image/png;base64,{encoded_image}" width="500" style="max-width: 100%;"/>'
                            download_link = f'<a href="data:image/png;base64,{encoded_image}" download="assistant_image.png">Download Image</a>'
                            image_elements.append(image_html + "<br>" + download_link)

                    # Display assistant's text response (Markdown processed)
                    if text_responses:
                        wrapped_text = "<br>".join(text_responses)
                        display(HTML(f'<div style="white-space: normal; word-wrap: break-word;">'
                                     f'<b>Assistant:</b> {wrapped_text}</div>'))

                    # Display images with download links
                    if image_elements:
                        display(HTML("<br>".join(image_elements)))

                    break  # Stop after handling the first assistant response

    else:
        with output_area:
            print("\nAssistant is processing your request. Please wait...")

    # Auto-focus input for next interaction
    auto_focus_input()

# Attach submit button click event
submit_button.on_click(handle_submit)

# Add Enter key functionality to the input field
user_input_widget.on_submit(handle_submit)

# Display UI components
display(output_area)
display(widgets.HBox([user_input_widget, submit_button]))

# Auto-focus input on load
auto_focus_input()
